In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
# Identify column headers
path = 'expanded_shuttle_three_days.csv'

import csv
with open(path, newline='') as f:
    reader = csv.reader(f)
    row1 = next(reader)

columns = row1

In [3]:
import pandas as pd

dtypes = {'SHUTTLE_COMPANY': str, 'RESTRICTION': str, 'ARTERIAL': str,
          'STREET': str}
df = pd.read_csv(path, error_bad_lines=False, names=columns, header=None, dtype=dtypes)

In [4]:
# Remove 1st roow
df = df.iloc[1:]

# Change type to datetime
df['TIMESTAMPLOCAL'] = pd.to_datetime(df['TIMESTAMPLOCAL'], format='%d-%b-%y %I.%M.%S.%f %p')

---

In [23]:
import sfmta
import json
from ipywidgets import Dropdown, HBox, VBox, Button, DatePicker, ColorPicker
from ipyleaflet import Map, Polyline, GeoJSON, Circle

def clear_map():
    if 'm' in globals():
        m.close()
    else:
        pass
    
def clear_lat_longs():
    if 'lat_longs' in globals():
        lat_longs = None
    else:
        pass

def on_button_clicked(b):
    clear_map()
    clear_lat_longs
    plate = plate_dropdown.value
    global lat_longs
    lat_longs = sfmta.lat_long_by_plate(df, plate)
    global m 
    m = draw_map(lat_longs)
    display(m)
    
button = Button(description="Draw/Update Map")
button.on_click(on_button_clicked)

def draw_map(lat_longs):
    center = [37.79481, -122.41186]
    zoom = 12

    m = Map(center=center, zoom=zoom)
    m.layout.height = '650px'
    pl = Polyline(locations=lat_longs)
    pl.color = path_color.value
    pl.fill_color = path_color.value
    m.add_layer(pl)
    return m

plates = df['LICENSE_PLATE_NUM'].unique()
plate_dropdown = Dropdown(options=plates, description='Plate')

# Add restrictions
def show_restrictions(b):
    polygons = 'vehiclerestrictions_wgs.json'
    
    with open(polygons) as f:
        polygons_json = json.load(f)

    global geojson
    geojson = GeoJSON(data=polygons_json)
    m.add_layer(geojson)
    

# Remove restrictions
def remove_restrictions(b):
    if 'geojson' in globals():
        m.remove_layer(geojson)
    else:
        pass
    
def download_data(b):
    if 'lat_longs' in globals():
        df = pd.DataFrame(data=lat_longs)
        df.to_excel('output.xlsx')
    else:
        pass

    
# # Add blocks
# def show_blocks(b):
#     blocks = 'tmp2.json'
    
#     with open(blocks) as f:
#         block_json = json.load(f)

#     global block_geojson
#     block_geojson = GeoJSON(data=block_json)
#     m.add_layer(block_geojson)

# button_blocks = Button(description="Show Blocks")
# button_blocks.on_click(show_blocks)

    
button_restrictions = Button(description="Show Restrictions")
button_restrictions.on_click(show_restrictions)
    
button_remove_restrictions = Button(description="Hide Restrictions")
button_remove_restrictions.on_click(remove_restrictions)

path_color = ColorPicker(description='Color')

export_data = Button(description="Download Data")
export_data.on_click(download_data)


def top_n_stops(b):
    from collections import Counter
    count_lat_longs = Counter(lat_longs).most_common()
    for lat_long in count_lat_longs[0:10]:
        location = lat_long[0]
        circle = Circle(location=location, radius=300)
        m.add_layer(circle)
    return m

button_add_top_stops = Button(description="Display Bus Idling")
button_add_top_stops.on_click(top_n_stops)

def remove_map(b):
    if 'm' in globals():
        m.close()
    else:
        pass

button_clear_map = Button(description="Remove Map")
button_clear_map.on_click(remove_map)

#Temp placeholders
import datetime
start = datetime.date(2017,11,6)
end = datetime.date(2017,11,9)
start_date = DatePicker(description='Start',value=start)
end_date = DatePicker(description='End', value=end)
companies = df['SHUTTLE_COMPANY'].unique()
company = Dropdown(options=companies, value='WeDriveU', description='Company')

VBox([HBox([company, button, button_restrictions, button_remove_restrictions]),
      HBox([plate_dropdown, export_data, button_add_top_stops, button_clear_map]),
      HBox([path_color]),
      HBox([start_date]),
      HBox([end_date])
     ])

VBox(children=(HBox(children=(Dropdown(description='Company', index=5, options=('Storer Transportation Service DBA Storer Coachways DBA Storer San Francisco', 'Lux Leasing LLC', 'Kaiser Permanente', 'Hallcon Corporation   (Loop Transportation, Inc.)', 'Pure Luxury Transportation', 'WeDriveU', 'Royal Coach Tours', 'Compass Transportation', 'TRANSMETRO', 'Corinthian International Parking Services Inc. ', 'Bishop Ranch Transportation', 'Black Tie Transportaion', 'San Francisco Minibus'), value='WeDriveU'), Button(description='Draw/Update Map', style=ButtonStyle()), Button(description='Show Restrictions', style=ButtonStyle()), Button(description='Hide Restrictions', style=ButtonStyle()))), HBox(children=(Dropdown(description='Plate', options=('CP96872', 'CP96879', '76734M1', '14053T1', '8T47570', '8Z97897', '8Z98015', '8Z98150', '60595U1', '48361F1', 'CP83137', 'PLBUS60', 'CP96880', 'CP80923', '96562J1', '8T69481', '8Z98048', 'CP96868', '63488B2', '63002E1', '23485S1', '02418T1', '63689A1', '93057W1', '23486S1', '8Z56334', '96263J1', '59956U1', '08236Y1', 'PLBUS73', '8T47569', 'PLBUS84', 'PLBUS57', '59955U1', '8T69480', 'CP96884', '93120W1', '23965S1', 'CP77577', nan, '31591E2', '58595C2', '8V53967', '62684X1', 'PLBUS76', 'CP96875', '96608J1', '8Z56587', 'EP00723', '44023L1', '93125W1', '8S75248', '64322B2', '8S63363', '15998C1', 'CP80515', '31588E2', '415640Z', '56055G1', '56054G1', '56056G1', '8V14304', '64943B2', '1654401', '23503U1', '87177R1', '8W02553', '15877V1', '16502C1', '98273k1', '93121W1', '84763A2', '44315L1', '63690A1', '63486B2', '63487B2', '96573J1', '8Z98043', '23502U1', '34763H2', '88358R1', '71947Z1', '04167T1', '08238Y1', '17447U1', '13861L1', '8Z98026', '11784N1', '03670T1', 'EP00725', '46984N1', '62686X1', '79508Y1', '64319B2', '08235Y1', '63301E1', '15336M1', '58368C2', '98084K1', '15925V1', '14237D2', '79505Y1', '03668T1', '04164T1', '84762A2', '02416T1', '83965D1', '88357R1', '08355Y1', '07757H1', '98387K1', '71946Z1', 'PLBUS56', '61862J1', '04165T1', '63687A1', '96403J1', 'BUJ6137', '98275k1', '8S75246', '69570X1', '8S63365', '96122J1', '58367C2', 'BUE3848', '78459N1', '98122S1', '69220X1', '63489B2', '51761B1', '34985H2', '8Y70385', '64320B2', '85844A2', '88360R1', '62679X1', '64097B2', '85651G1', '53600P1', 'EP00710', '66225X1', '98272k1', '65270V1', '48042S1', '96618J1', '80696N1', '8Z90217', '63142E1', '34764H2', '80693N1', '8Z56585', '61740B2', '96405J1', '71921Z1', '64079B2', '85513G1', '96480J1', '79506Y1', 'EP00711', '81199Y1', '96219J1', '64942B2', '58307C2', '66703X1', '85628G1', '58369C2', '96574J1', '64383B2', '60497U1', '62678X1', '33771A2', '08719Y1', '66230X1', '30689E2', '65274V1', '09469D2', '85785G1', '65275V1', '87178R1', '35368K1', '96402J1', '92396W1', '92571W1', '66957V1', '98778E2', '66484V1', 'EP00724', '80694N1', '63688A1', '87181R1', '8Z98046', '34987H2', '96406J1', '71922Z1', '35021H2', '92363W1', '8Y70386', '16543E1', '92361W1', '48362F1', '16280V1', 'CP80825', '09803D2', '33195A2', '08354Y1', '8Z97907', 'EP00709', '33193A2', '14326P1', '8Y70294', '08352Y1', 'CP96874', '23501U1', '30692E2', 'EP00726', '03603T1', '52035B1', '48382F1', '92364W1', '85787G1', '62683X1', '03669T1', '87180R1', '59410T1', '09500D2', '85784G1', '08237Y1', '16503C1', '09502D2', '14302D2', '98083K1', '68932X1', '33194A2', '88355R1', 'EP00708', '62677X1', '23207U1', '63295E1', '02455T1', '13859L1', '69569X1', '31975T1', '15876V1', '63584E1', '78461N1', '69568X1', '15878V1', '16500C1', '07851H1', '66222V1', '62681X1', '23504U1', '59499C2', '8Z56577', '92362W1', '62675X1', '19021U1', '34986H2', '48043S1', '93126W1', '58366C2', '66221V1', '62685X1', '59411T1', '61715J1', '62687X1', '30691E2', '79507Y1', '63719E1', '83863A2', '59959U1', '63486A1', '34988H2', '59498C2', '63908B2', '85443G1', 'PLBUS77', 'CP80824', '15337M1', '66223X1', '92452W1', '61499B2', '8Z97821', '8Z97908', '48381F1', '13863L1', '76421Y1', '96265J1', '09715D2', '59608U1', '96302J1', '95105V1', '96616J1', 'EP00707', '76512Y1', '30688E2', '

In [ ]:
center = [37.79481, -122.41186]
zoom = 12

m2 = Map(center=center, zoom=zoom)
m2

In [ ]:
m2.add_layer(block_geojson)
m2

In [ ]:
m2.clear_layers()

In [ ]:
#print(block_geojson)

In [ ]:
block_polygons = 'block_polygons.json'
block_shape_file = '../StreetPolygons/StreetPolygons.shp'

In [ ]:
import geopandas as gpd

In [ ]:
block_polygons_df = gpd.read_file(block_shape_file)
block_polygons_df.head()

In [ ]:
columns = ['CNN', 'geometry', 'SHAPE_AREA']
columns2 = ['geometry']
block_df = block_polygons_df[columns2]
block_df.to_file(filename='tmp2.json', driver='GeoJSON')

In [ ]:
block_polygons_df.iloc[4].geometry

In [ ]:
import fiona; fiona.supported_drivers